In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

ratings = pd.read_csv('/content/drive/MyDrive/ratings.csv', sep='\t', encoding='latin-1',
                      usecols=['user_id', 'movie_id', 'user_emb_id', 'movie_emb_id', 'rating'])

movies = pd.read_csv('/content/drive/MyDrive/movies.csv', sep='\t', encoding='latin-1',
                     usecols=['movie_id', 'title', 'genres'])

In [ ]:
movies.head(5)

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
ratings.head(5)

,user_id,movie_id,rating,user_emb_id,movie_emb_id
0,1,1193,5,0,1192
1,1,661,3,0,660
2,1,914,3,0,913
3,1,3408,4,0,3407
4,1,2355,5,0,2354


### Content Based

**Objective**: Build a Content-Based Recommendation system that computes similarity between movies based on movie genres. It will suggest movies that are most similar to a particular movie based on its genre.

**Dataset**:
- **Movies Data**: DataFrame with columns `movie_id`, `title`, and `genres`.

**Steps**:

1. **Preprocess Data**:
   - Split the `genres` column into separate genre strings.
   - Fill any missing values in the `genres` column and convert it to string.

2. **Compute TF-IDF Matrix**:
   - Use `TfidfVectorizer` to transform the genres into a TF-IDF matrix.

3. **Calculate Cosine Similarity**:
   - Compute cosine similarity between the TF-IDF matrices of movies.

4. **Recommend Movies**:
   - Create a function that gets movie recommendations based on the cosine similarity score of movie genres.

In [ ]:
movies['genres'] = movies['genres'].str.split('|')
movies['genres'] = movies['genres'].fillna("").astype('str')

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(movies['genres'])

In [ ]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
unique_movies = movies.drop_duplicates(subset='title', keep='first')
indices = pd.Series(unique_movies.index, index=unique_movies['title'])

def genre_recommendations(title, n=10):
    similarities = enumerate(cosine_sim[indices[title]])

    sorted_similarities = sorted(similarities, key=lambda x: x[1], reverse=True)[1:n+1]

    recommended_indices = [index for index, score in sorted_similarities]

    recommended_titles = []

    for index in recommended_indices:
        row = movies.iloc[index]

        title = row['title']

        recommended_titles.append(title)

    return recommended_titles





In [ ]:
# Example
genre_recommendations('GoldenEye (1995)')

['Clear and Present Danger (1994)',
 'Surviving the Game (1994)',
 'Rock, The (1996)',
 'Daylight (1996)',
 'Chain Reaction (1996)',
 'Maximum Risk (1996)',
 'Anaconda (1997)',
 'Con Air (1997)',
 'Firestorm (1998)',
 'Perfect Storm, The (2000)']


### Collaborative Filtering

**Objective**: Implement a collaborative filtering movie recommendation system to recommend movies to a user based on similar users' spreferences.

**Dataset**:

- **Ratings Data**: DataFrame with columns `user_id`, `movie_id`, and `rating`.
- **Movies Data**: DataFrame with columns `movie_id`, `title`, and `genres`.

**Steps**:

1. **Preprocess Data**:
   - Merge `ratings` and `movies` DataFrames on `movie_id`.
   - Split `genres` column into separate rows.

2. **Create User-Genre Matrix**:
   - Create a matrix where rows are users and columns are genres.
   - Each cell represents the ratio of movies watched by the user in that genre to the total movies watched by the user.

3. **Calculate User Similarity**:
   - Use cosine similarity to calculate the similarity between users.

4. **Find Top N Similar Users**:
   - Find the top N users with the most similar preferences to the given user.

5. **Recommend Movies**:
   - Recommend movies that similar users have watched but the target user has not.
   - Return titles and genres of the top 10 recommended movies.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
data = pd.merge(ratings, movies, on='movie_id')

In [ ]:
data.head(5)

,user_id,movie_id,rating,user_emb_id,movie_emb_id,title,genres
0,1,1193,5,0,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']
1,2,1193,5,1,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']
2,12,1193,4,11,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']
3,15,1193,4,14,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']
4,17,1193,5,16,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']


In [ ]:
data = data.assign(genres=data['genres'].str.split('|')).explode('genres')

In [ ]:
data.head(5)

,user_id,movie_id,rating,user_emb_id,movie_emb_id,title,genres
0,1,1193,5,0,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']
1,2,1193,5,1,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']
2,12,1193,4,11,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']
3,15,1193,4,14,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']
4,17,1193,5,16,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']


In [ ]:
user_genre_matrix = pd.crosstab(index=data['user_id'], columns=data['genres'])

user_genre_matrix_normalized = user_genre_matrix.div(user_genre_matrix.sum(axis=1), axis=0)

user_similarity = cosine_similarity(user_genre_matrix_normalized)
user_similarity_df = pd.DataFrame(user_similarity, index=user_genre_matrix.index, columns=user_genre_matrix.index)


def get_top_n_similar_users(user_id, n=5):
    sorted_scores = user_similarity_df.loc[user_id].nlargest(n + 1)
    top_users = sorted_scores.index[1:]
    return top_users


In [ ]:
def recommend_movies(user_id, n_similar_users=5, n_recommendations=10):

    user_seen_movies = set(data[data['user_id'] == user_id]['movie_id'])

    recommendations = data[data['user_id'].isin(get_top_n_similar_users(user_id, n_similar_users)
)][~data[data['user_id'].isin(get_top_n_similar_users(user_id, n_similar_users)
)]['movie_id'].isin(user_seen_movies)]

    top_recommendations = recommendations['movie_id'].value_counts().nlargest(n_recommendations).index

    recommended_movies = movies[movies['movie_id'].isin(top_recommendations)][['title', 'genres']]

    return recommended_movies

In [ ]:
# Example
user_id = 1
recommended_movies = recommend_movies(user_id, n_similar_users=5, n_recommendations=10)

In [ ]:
recommended_movies

,title,genres
589,"Silence of the Lambs, The (1991)","['Drama', 'Thriller']"
592,Pinocchio (1940),"['Animation', ""Children's""]"
1019,Alice in Wonderland (1951),"['Animation', ""Children's"", 'Musical']"
1207,Amadeus (1984),['Drama']
1254,Akira (1988),"['Adventure', 'Animation', 'Sci-Fi', 'Thriller']"
2011,Lady and the Tramp (1955),"['Animation', ""Children's"", 'Comedy', 'Musical..."
2027,Sleeping Beauty (1959),"['Animation', ""Children's"", 'Musical']"
2789,American Beauty (1999),"['Comedy', 'Drama']"
3091,Magnolia (1999),['Drama']
3412,High Fidelity (2000),['Comedy']


### SVD (Singular Value Decomposition)



**Objective**: Implement an SVD-based recommendation system to recommend movies to users by decomposing the user-item interaction matrix into latent factors.

**Dataset**:
- **Ratings Data**: DataFrame with columns `user_id`, `movie_id`, and `rating`.
- **Movies Data**: DataFrame with columns `movie_id`, `title`, and `genres`.

**Steps**:

1. **Preprocess Data**:
   - Merge `ratings` and `movies` DataFrames on `movie_id`.
   - Create a user-item matrix where rows represent users, columns represent movies, and the values are the ratings.

2. **Decompose Matrix using SVD**:
   - Apply Singular Value Decomposition (SVD) to decompose the user-item matrix into three matrices: $U$, $\Sigma$, and $V^T$.

3. **Reconstruct Matrix**:
   - Reconstruct the user-item matrix using the top $k$ singular values to reduce dimensionality.

4. **Predict Ratings**:
   - Use the reconstructed matrix to predict ratings for all user-item pairs.

5. **Recommend Movies**:
   - Recommend the top 20 movies with the highest predicted ratings for a given user that the user hasn't rated yet.

In [ ]:
n_users = ratings.user_id.unique().shape[0]
n_movies = ratings.movie_id.unique().shape[0]
print('Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_movies))

Number of users = 6040 | Number of movies = 3706


Fill na elements.

In [ ]:
Ratings = ratings.pivot(index = 'user_id', columns ='movie_id', values = 'rating').fillna(0)
Ratings.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from scipy.sparse.linalg import svds

In [ ]:
U, sigma, Vt = svds(np.array(Ratings), k=840)
sigma = np.diag(sigma)

In [ ]:
all_user_predicted_ratings = np.einsum('ik,kj->ij', U @ sigma, Vt)


In [ ]:
preds = pd.DataFrame(all_user_predicted_ratings, columns=Ratings.columns)
preds.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
0,4.980888,0.090600,0.049229,0.251474,-0.065108,-0.117803,0.021080,-0.195696,-0.005885,-0.227240,...,0.056215,-0.064958,0.048049,-0.116143,0.020263,-0.022996,-0.112011,-0.017384,-0.041587,-0.239509
1,0.070783,0.248592,0.166922,-0.036630,0.057151,0.075751,-0.062135,0.154114,0.100605,0.211973,...,0.000758,0.016206,-0.073841,0.084230,0.072769,-0.380425,0.029762,0.031551,-0.041977,-0.269854
2,0.077053,0.224685,0.029751,0.300464,-0.103837,0.158163,0.091336,-0.067043,0.016575,0.318254,...,0.060825,0.002622,0.067589,0.015823,0.037632,0.065567,-0.109586,0.088560,0.008233,-0.240962
3,0.053475,0.144704,0.018034,-0.100595,0.216529,0.042516,0.135589,-0.005337,0.039234,0.045281,...,0.004157,0.039170,0.030833,-0.077997,-0.002048,-0.060750,0.117341,0.055521,-0.029243,-0.166704
4,0.046517,-0.844449,-0.281974,0.103162,-0.127785,1.759876,0.369107,0.096935,0.015354,0.072317,...,0.328407,0.012128,-0.088363,-0.163630,-0.200876,0.047539,0.242808,-0.021348,-0.232320,-0.496233


In [ ]:
import pandas as pd

def recommend_movies(predictions, userID, movies, original_ratings, num_recommendations):
    user_row_number = userID - 1



    user_row_number = userID - 1

    user_predictions = predictions.iloc[user_row_number]
    sorted_user_predictions = user_predictions.sort_values(ascending=False)

    user_data = original_ratings[original_ratings['user_id'] == userID]

    user_full = pd.merge(user_data, movies, how='left', on='movie_id').sort_values(by='rating', ascending=False)

    print(f'User {userID} has already rated {user_full.shape[0]} movies.')
    print(f'Recommending the highest {num_recommendations} predicted rating movies not already rated.')

    unrated_movies = movies[~movies['movie_id'].isin(user_full['movie_id'])]

    recommendations = unrated_movies.merge(pd.DataFrame(sorted_user_predictions).reset_index(), how='left',
                                          left_on='movie_id', right_on='movie_id')
    recommendations = recommendations.rename(columns={user_row_number: 'Predictions'}).sort_values(by='Predictions', ascending=False)
    recommendations = recommendations.iloc[:num_recommendations, :-1]

    return user_full, recommendations

In [ ]:
already_rated, predictions = recommend_movies(preds, 4375, movies, ratings, 20)

User 4375 has already rated 325 movies.
Recommending the highest 20 predicted rating movies not already rated.


In [ ]:
ratings.head(20)

,user_id,movie_id,rating,user_emb_id,movie_emb_id
0,1,1193,5,0,1192
1,1,661,3,0,660
2,1,914,3,0,913
3,1,3408,4,0,3407
4,1,2355,5,0,2354
5,1,1197,3,0,1196
6,1,1287,5,0,1286
7,1,2804,5,0,2803
8,1,594,4,0,593
9,1,919,4,0,918


In [ ]:
# Top 20 movies that User 4375 has rated
already_rated.head(20)

,user_id,movie_id,rating,user_emb_id,movie_emb_id,title,genres
195,4375,3250,5,4374,3249,Alive (1993),['Drama']
285,4375,175,5,4374,174,Kids (1995),['Drama']
199,4375,3257,5,4374,3256,"Bodyguard, The (1992)","['Action', 'Drama', 'Romance', 'Thriller']"
65,4375,3809,5,4374,3808,What About Bob? (1991),['Comedy']
210,4375,2688,5,4374,2687,"General's Daughter, The (1999)","['Drama', 'Thriller']"
270,4375,150,5,4374,149,Apollo 13 (1995),['Drama']
271,4375,2710,5,4374,2709,"Blair Witch Project, The (1999)",['Horror']
125,4375,349,5,4374,348,Clear and Present Danger (1994),"['Action', 'Adventure', 'Thriller']"
60,4375,288,5,4374,287,Natural Born Killers (1994),"['Action', 'Thriller']"
141,4375,47,5,4374,46,Seven (Se7en) (1995),"['Crime', 'Thriller']"


#### Evaluation

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

train_data, test_data = train_test_split(ratings, test_size=0.2, random_state=42)

train_ratings = train_data.pivot(index='user_id', columns='movie_id', values='rating').fillna(0)
R_train = train_ratings.values
user_ratings_mean_train = np.mean(R_train, axis=1)
Ratings_demeaned_train = R_train - user_ratings_mean_train.reshape(-1, 1)

U_train, sigma_train, Vt_train =  svds(Ratings_demeaned_train, k =840)
sigma_train = np.diag(sigma_train)


intermediate_result = np.dot(U_train, sigma_train)
final_result = np.dot(intermediate_result, Vt_train)
all_user_predicted_ratings_train = final_result + user_ratings_mean_train[:, np.newaxis]



preds_train = pd.DataFrame(all_user_predicted_ratings_train, columns=train_ratings.columns)


def predict_rating(user_id, movie_id):
    try:
        return preds_train.at[user_id-1, movie_id]
    except KeyError:
        return float('nan')

test_data['predicted_rating'] = test_data.apply(lambda x: predict_rating(x['user_id'], x['movie_id']), axis=1)

test_data.dropna(inplace=True)

rmse = sqrt(mean_squared_error(test_data['rating'], test_data['predicted_rating']))
print(f'Root Mean Squared Error: {rmse}')


Root Mean Squared Error: 3.6454182145573983


### Nueral Network Model (Recommender Model)



**Objective**: Implement a Recommender model to recommend movies to a user based on similar users' preferences.

**Dataset**:
- **Ratings Data**: DataFrame with columns `user_id`, `movie_id`, and `rating`.
- **Movies Data**: DataFrame with columns `movie_id`, `title`, and `genres`.

**Steps**:

1. **Define Dataset and DataLoader**:
   - Create a custom PyTorch `Dataset` for ratings.
   - Create a DataLoader for batching and shuffling data.

2. **Define the Neural Network**:
   - Create a neural network with embedding layers for users and movies.

3. **Train the Model**:
   - Train the model using Mean Squared Error loss and Adam optimizer.
   - Save model checkpoints.

4. **Evaluate the Model**:
   - Calculate RMSE on the entire dataset.

5. **Predict Ratings for Unrated Movies**:
   - Predict and recommend top 10 unrated movies for a given user.

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import math

In [ ]:
# Define the dataset
class RatingsDataset(Dataset):
    def __init__(self, ratings):
        self.user_ids = ratings['user_id'].values
        self.ratings = ratings['rating'].values
        self.movie_ids = ratings['movie_id'].values

    def __len__(self):
        return len(self.user_ids)

    def __getitem__(self, idx):
        return self.user_ids[idx], self.movie_ids[idx], self.ratings[idx]

# Define the neural network
class RecommenderNet(nn.Module):

    def __init__(self, num_users, num_movies, embedding_size=50):
        super(RecommenderNet, self).__init__()
        self.user_embedding = self.create_embedding(num_users, embedding_size)
        self.movie_embedding = self.create_embedding(num_movies, embedding_size)
        self.fc = self.create_linear_layer(embedding_size * 2, 1)

    def forward(self, user, movie):
        user_embedded = self.user_embedding(user)
        movie_embedded = self.movie_embedding(movie)
        concatenated = self.concat_embeddings(user_embedded, movie_embedded)
        output = self.fc(concatenated)
        return output.squeeze()

    def create_embedding(self, num_items, embedding_size):
        return nn.Embedding(num_items, embedding_size)

    def create_linear_layer(self, in_features, out_features):
        return nn.Linear(in_features, out_features)

    def concat_embeddings(self, embed1, embed2):
        return torch.cat((embed1, embed2), dim=1)


# Create the dataset and dataloader
ratings_dataset = RatingsDataset(ratings)
data_loader = DataLoader(ratings_dataset, batch_size=64, shuffle=True)

# Initialize the model, loss function, and optimizer
num_users = len(set(ratings['user_id']))
num_movies = len(set(ratings['movie_id']))
model = RecommenderNet(num_users+1000, num_movies+1000)
optimizer = optim.Adam(model.parameters(), lr=0.0005)
criterion = nn.MSELoss()

# Training loop
num_epochs = 15
max_iterations=1000
for epoch in range(num_epochs):
    for i , (user, movie, rating) in enumerate(data_loader):
        optimizer.zero_grad()
        output = model(user, movie)
        loss = criterion(output, rating.float())
        loss.backward()
        optimizer.step()

        if i == max_iterations:
          break
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')


# Evaluation

def evaluate_model(model, dataloader):

    model.eval()
    predictions=[]
    true_lables = []

    with torch.no_grad():
        for user, movie, rating in dataloader:
            output = model(user, movie)
            predictions += output.tolist()
            true_lables +=rating.tolist()


            #true_lables.extend(rating.tolist())

    rmse= math.sqrt(sum([(pred - actual)**2 for pred, actual in zip(predictions, true_lables)]) / len(true_lables))

    return rmse

# Example usage:
train_rmse = evaluate_model(model, data_loader)
print('Train RMSE is ',train_rmse)


Epoch 1, Loss: 7.053975582122803
Epoch 2, Loss: 2.06005597114563
Epoch 3, Loss: 1.7874372005462646
Epoch 4, Loss: 1.622281551361084
Epoch 5, Loss: 1.4952409267425537
Epoch 6, Loss: 1.1066926717758179
Epoch 7, Loss: 0.9419023394584656
Epoch 8, Loss: 0.9385327100753784
Epoch 9, Loss: 1.1691784858703613
Epoch 10, Loss: 0.9600309133529663
Epoch 11, Loss: 0.6162116527557373
Epoch 12, Loss: 0.7303176522254944
Epoch 13, Loss: 1.2479639053344727
Epoch 14, Loss: 0.9757030606269836
Epoch 15, Loss: 0.9469412565231323
Train RMSE is  0.9243153017652312


In [ ]:
# Example

sample_user_id = 1
sample_user_data = ratings[ratings['user_id'] == sample_user_id]
sample_dataset = RatingsDataset(sample_user_data)
sample_dataloader = DataLoader(sample_dataset, batch_size=1, shuffle=False)

print(f'Sample predictions for user ID {sample_user_id}:')
with torch.no_grad():
    for user, movie, rating in sample_dataloader:
        output = model(user, movie).squeeze()
        print(f'Movie ID: {movie.item()}, Predicted Rating: {output.item()}, Actual Rating: {rating.item()}')

Sample predictions for user ID 1:
Movie ID: 1193, Predicted Rating: 4.770419597625732, Actual Rating: 5
Movie ID: 661, Predicted Rating: 4.025676250457764, Actual Rating: 3
Movie ID: 914, Predicted Rating: 4.53526496887207, Actual Rating: 3
Movie ID: 3408, Predicted Rating: 4.216932773590088, Actual Rating: 4
Movie ID: 2355, Predicted Rating: 4.315035820007324, Actual Rating: 5
Movie ID: 1197, Predicted Rating: 4.684384346008301, Actual Rating: 3
Movie ID: 1287, Predicted Rating: 4.494870185852051, Actual Rating: 5
Movie ID: 2804, Predicted Rating: 4.752345561981201, Actual Rating: 5
Movie ID: 594, Predicted Rating: 4.1231369972229, Actual Rating: 4
Movie ID: 919, Predicted Rating: 4.659842014312744, Actual Rating: 4
Movie ID: 595, Predicted Rating: 4.344738006591797, Actual Rating: 5
Movie ID: 938, Predicted Rating: 3.8717288970947266, Actual Rating: 4
Movie ID: 2398, Predicted Rating: 4.21862268447876, Actual Rating: 4
Movie ID: 2918, Predicted Rating: 4.596291542053223, Actual Ratin

In [ ]:
def predict_unrated_movies(user_id, model, ratings, movies):

    user_rated_movies = ratings[ratings['user_id'] == user_id]['movie_id']
    user_unrated_movies = movies[~movies['movie_id'].isin(user_rated_movies)]

    user_ids = torch.full((len(user_unrated_movies),), user_id, dtype=torch.long)
    movie_ids = torch.tensor(user_unrated_movies['movie_id'].values, dtype=torch.long)

    with torch.no_grad():
        ratings = model(user_ids, movie_ids)

    user_unrated_movies = user_unrated_movies.copy()  # Create a copy
    user_unrated_movies.loc[:, 'predicted_rating'] = ratings.numpy()
    recommended_movies = user_unrated_movies.nlargest(10, 'predicted_rating')

    return recommended_movies



# Example
user_id = 1
predictions = predict_unrated_movies(user_id, model, ratings, movies)
print(predictions)

      movie_id                                              title  \
2754      2823  Spiders, The (Die Spinnen, 1. Teil: Der Golden...   
2886      2955                             Penitentiary II (1982)   
1090      1106                            Leopard Son, The (1996)   
2862      2931        Time of the Gypsies (Dom za vesanje) (1989)   
863        874                 Killer: A Journal of Murder (1995)   
2836      2905                                     Sanjuro (1962)   
1180      1198                     Raiders of the Lost Ark (1981)   
847        858                              Godfather, The (1972)   
1950      2019  Seven Samurai (The Magnificent Seven) (Shichin...   
3238      3307                                 City Lights (1931)   

                              genres  predicted_rating  
2754             ['Action', 'Drama']          5.530055  
2886                       ['Drama']          5.262606  
1090                 ['Documentary']          5.226938  
2862        

### GMM (Gaussian Mixture Model)

**Objective**: Use a Gaussian Mixture Model to analyze and cluster the click data based on the number of clicks from different locations, aiming to identify distinct patterns of user behavior across 10 countries.

**Dataset**:
- **Click Data**: DataFrame with columns `link_id`, `location`, and `number_of_clicks`.

**Steps**:

1. **Preprocess Data**:
   - Ensure the dataset contains 10 distinct countries.
   - Create a matrix where rows represent different links and columns represent the number of clicks from each country.
   - Normalize the number of clicks to account for different scales.

2. **Fit GMM**:
   - Apply a Gaussian Mixture Model (GMM) to the click data matrix to identify clusters of links with similar click patterns across different countries.

3. **Evaluate Model**:
   - Analyze the resulting model by calculating the log lokelihood, BIC and AIC metrics.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

file_path = '/content/drive/MyDrive/gmm-dataset.csv'
df = pd.read_csv(file_path)

urls = df.loc[:, df.columns[0]].to_numpy()
click_data = df.loc[:, df.columns[1]:].to_numpy()


normalized_clicks = StandardScaler().fit_transform(click_data)

num_clusters = 11

gmm = GaussianMixture(n_components=num_clusters, random_state=0)

gmm.fit(normalized_clicks)

labels = gmm.predict(normalized_clicks)

silhouette_avg = silhouette_score(normalized_clicks, labels)

log_likelihood = gmm.score(normalized_clicks)
bic = gmm.bic(normalized_clicks)
aic = gmm.aic(normalized_clicks)

print(f"Number of clusters: {num_clusters}")
print(f"Silhouette Score: {silhouette_avg:.4f}")
print(f"Log-Likelihood: {log_likelihood:.4f}")
print(f"BIC: {bic:.4f}")
print(f"AIC: {aic:.4f}")

Number of clusters: 11
Silhouette Score: 0.1104
Log-Likelihood: 0.0700
BIC: 3324.7552
AIC: 1436.0068
